# Overview of GloFas Data Processing Notebook

## Introduction

This notebook demonstrates the process of accessing, downloading, and processing GloFas (Global Flood Awareness System) data for flood forecasting and analysis using the mcimageprocessing package.

## Objective

The main objective of this notebook is to provide a practical example of how to programmatically:

1. Set up environment configurations for data processing.
2. Load and manipulate geospatial boundaries using GeoPandas.
3. Interact with Google Earth Engine through an API to enhance data context.
4. Download, process, and clip GloFas data to focus on specific areas of interest.
5. Prepare the data for further analysis or visualization tasks.

## Workflow

The workflow of the notebook is structured as follows:
- **Setting Up Environment Variables**: Configure the paths necessary for the APIs and processing tools.
- **Importing Modules**: Load all necessary Python libraries and custom modules required for processing.
- **Loading Geospatial Data**: Read the area of interest defined in a GeoJSON file.
- **Initializing Data Managers**: Set up connections to data sources like Google Earth Engine.
- **Generating Bounding Box**: Generate a bounding box from the GeoDataFrame to define the area of interest for GloFas data retrieval.
- **Retrieving and Processing Data**:
    - Set parameters for data retrieval based on GloFas product specifics.
    - Download the relevant GloFas data files for the defined time and area.
    - Clip the downloaded data to the boundaries of the specified area of interest.
- **Post-Processing**:
    - Prepare the processed files for analysis, merging if necessary, and ensure they are in a usable format (e.g., TIFF).


## Step 1: Setting Up Environment Variables

### Set the required environment variable for configuration directory. Alternatively, if not set, when mcimageprocessing is imported, it will request the user define a path and subsequently set the path to the config file.

In [ ]:
import os

# Set environment variable
os.environ['CONFIG_DIR'] = # SET PATH TO CONFIG DIRECTORY

folder_path = # SET FOLDER PATH FOR DOWNLOADS/PROCESSING


## Step 2: Importing Necessary Modules

### Import the modules needed for processing GloFas data.

In [ ]:
from mcimageprocessing.programmatic.APIs.EarthEngine import EarthEngineManager
from mcimageprocessing.programmatic.APIs.GloFasAPI import GloFasAPI
from mcimageprocessing.jupyter.JupyterAPI import JupyterAPI
from mcimageprocessing.programmatic.shared_functions.utilities import generate_bbox, clip_raster
import json
import geopandas as gpd

## Step 3: Loading Geospatial Data

### Load geojson geometry to define the area of interest.

In [ ]:
with open('geometry.geojson', 'r') as fp:
    geometry_json = json.load(fp)
gdf = gpd.GeoDataFrame.from_features(geometry_json['features'])

## Step 4: Initializing GloFas Data Manager

### Initialize the GloFas data manager with Earth Engine.

In [ ]:
glofas = GloFasAPI(ee_manager=EarthEngineManager())

## Step 5: Generating Bounding Box

### Generate a bounding box from the GeoDataFrame.

In [ ]:
bbox = generate_bbox(gdf)

## Step 6: Set Params

### Set the params for glofas. This can either be static, or can be where looping and/or automation logic can be implemented. For instance, if you are interested in multiple days or months, have this embedded in a for loop with lists of months/days, and then include the glofas.download_glofas_data function and clip function inside the loop.

In [ ]:
params = {
    'glofas_product': 'cems-glofas-forecast',
    'system_version': 'operational',
    'hydrological_model': 'lisflood',
    'product_type': 'ensemble_perturbed_forecasts',
    "variable": "river_discharge_in_the_last_24_hours",
    'year': 2023,
    'month': '04',
    'day': '01',
    'leadtime_hour': [24], 
    'area': [bbox['miny'][0],
            bbox['minx'][0],
            bbox['maxy'][0],
            bbox['maxx'][0]], 
    'folder_location': folder_path,
    "format": "grib"
}


## Step 7: Download GloFas Data


In [ ]:
try:
    file_path = glofas.download_glofas_data(bbox=bbox, params=params)
    print(f"Data downloaded successfully: {file_path}")
except Exception as e:
    print(f"An error occurred: {str(e)}")


## Step 8: Clipping the Raster File

### Clip the merged raster file to the boundary of the GeoDataFrame.

In [ ]:
clip_raster(file_path, gdf)